In [5]:
from datasets import load_dataset
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')
train_df = pd.read_pickle("/content/drive/My Drive/CMPE 257/CMPE 257 Colab/257 Sensitive Data Input Guardrail/train_df.pkl")
test_df = pd.read_pickle("/content/drive/My Drive/CMPE 257/CMPE 257 Colab/257 Sensitive Data Input Guardrail/test_df.pkl")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Random Forest Classifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


# vectorize text with TF-IDF
vectorizer = TfidfVectorizer(max_features=1500, stop_words="english")

X_train = vectorizer.fit_transform(train_df["source_text"])
X_test = vectorizer.transform(test_df["source_text"])
y_train = train_df["is_sensitive"]
y_test = test_df["is_sensitive"]


# best parameters overall according to gridsearch, but context here matters
# with sensitive data, it is paramount that we have near 100% recall
# this random forest below has a recall < .9
rf = RandomForestClassifier(n_estimators = 404, max_depth = 48, min_samples_split = 3, min_samples_leaf = 3, max_features = 'sqrt', n_jobs = -1, random_state = 42, class_weight = 'balanced')
rf.fit(X_train, y_train)


# evaluate
y_pred = rf.predict(X_test)

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=3))


feature_names = vectorizer.get_feature_names_out()
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1][:20]

print("\nTop 20 Important Words for Sensitive Detection:")
for i in indices:
    print(f"{feature_names[i]}: {importances[i]:.4f}")


Confusion Matrix:
[[ 805  103]
 [ 956 6082]]

Classification Report:
              precision    recall  f1-score   support

           0      0.457     0.887     0.603       908
           1      0.983     0.864     0.920      7038

    accuracy                          0.867      7946
   macro avg      0.720     0.875     0.762      7946
weighted avg      0.923     0.867     0.884      7946


Top 20 Important Words for Sensitive Detection:
com: 0.0512
number: 0.0332
passport: 0.0292
username: 0.0243
id: 0.0227
address: 0.0193
road: 0.0187
ip: 0.0187
11: 0.0186
date: 0.0180
00: 0.0177
eng: 0.0176
pm: 0.0156
license: 0.0146
password: 0.0130
street: 0.0124
time: 0.0123
sex: 0.0122
12: 0.0120
email: 0.0117


In [7]:
# pip install optuna

In [8]:
# Takes like 1.5 hours to run
# Best Params: {'n_estimators': 404, 'max_depth': 48, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 'sqrt'}
# Best F1 Score: 0.918767248955558


# import optuna
# from sklearn.model_selection import cross_val_score

# def objective(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 500),
#         'max_depth': trial.suggest_int('max_depth', 5, 50),
#         'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
#         'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
#         'class_weight': 'balanced',
#         'random_state': 42
#     }
#     rf = RandomForestClassifier(**params)
#     f1 = cross_val_score(rf, X_train, y_train, scoring='f1', cv=3, n_jobs=-1).mean()
#     return f1

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=30)

# print("Best Params:", study.best_params)
# print("Best F1 Score:", study.best_value)

